In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
def describir_shp(gdf):
    area_cargado = gdf['area'].sum()
    gdf['area'] = gdf.geometry.area / 10000
    area_recalculado = gdf['area'].sum()
    gdf['idd'] = gdf.apply(lambda row: f"{row['unidad_01']}|{row['unidad_03']}|{row['unidad_05']}", axis=1)
    gdf_disolve = gdf.dissolve(by='idd')
    gdf_disolve['area'] = gdf_disolve.geometry.area / 10000
    area_disolve = gdf_disolve['area'].sum()
    estado_datos = {
        'Items' : ['Area cargado', 'Area recalculado', 'Area disolve'],
        'Datos' : [round(area_cargado, 2), round(area_recalculado, 2), round(area_disolve, 2)]
    }
    return pd.DataFrame(estado_datos)

In [4]:
path_shp_cosecha = r'G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - CATASTRO/Catastro/CATASTRO 2024/cosecha2024rv.shp'
path_shp_catastro = r'G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - CATASTRO/Catastro/CATASTRO 2024/catastro080824.shp'

In [5]:
gdf_cosecha = gpd.read_file(path_shp_cosecha)
gdf_catastro = gpd.read_file(path_shp_catastro)

In [6]:
gdf_cosecha['unidad_01'] = gdf_cosecha['unidad_01'].astype(int)
gdf_catastro['unidad_01'] = gdf_catastro['unidad_01'].astype(int)

In [7]:
describir_shp(gdf_cosecha)

,Items,Datos
0,Area cargado,53918.63
1,Area recalculado,53918.63
2,Area disolve,53918.56


In [8]:
gdf_catastro_canha = gdf_catastro[gdf_catastro['cultivo']=='canha'].copy()

In [9]:
describir_shp(gdf_catastro_canha)

,Items,Datos
0,Area cargado,54109.41
1,Area recalculado,54112.45
2,Area disolve,54112.45


In [10]:
def comprobar_idd_repetidos(gdf):
    gdf['idd'] = gdf.apply(lambda row: f"{row['unidad_01']}|{row['unidad_03']}|{row['unidad_05']}", axis=1)
    gdf_orden = gdf.sort_values(by='idd')
    conteos = gdf['idd'].value_counts()
    valores_repetidos = conteos[conteos > 1]
    return valores_repetidos

# COMPROBAR CATASTRO

In [11]:
gdf_catastro_canha['unidad_03']

0        42047
1         2913
2         8816
3        41967
4        42047
         ...  
13413      805
13414      805
13415    18027
13416     2565
13417    11703
Name: unidad_03, Length: 10637, dtype: int64

In [12]:
gdf_catastro_canha['unidad_03'] = gdf_catastro_canha['unidad_03'].astype(int)

In [13]:
comprobar_idd_repetidos(gdf_catastro_canha)

idd
129|8028|L2        2
1979|41954|L1      2
184|14562|L28.2    2
613|2615|L3        2
299|1530|S7        2
129|8028|L1.1      2
Name: count, dtype: int64

In [14]:
gdf_catastro_disolve = gdf_catastro_canha.dissolve(by='idd')
gdf_catastro_disolve['area'] = gdf_catastro_disolve.geometry.area / 10000
gdf_catastro_disolve = gdf_catastro_disolve.reset_index()
area_catastro_disolve = gdf_catastro_disolve['area'].sum()

In [15]:
gdf_catastro_disolve['unidad_03'] = gdf_catastro_disolve['unidad_03'].astype(int)

In [16]:
area_catastro_disolve

54112.448550171815

In [17]:
comprobar_idd_repetidos(gdf_catastro_disolve)

Series([], Name: count, dtype: int64)

# COMPROBAR SHP COSECHA

In [18]:
gdf_cosecha_disolve = gdf_cosecha.dissolve(by='idd')
gdf_cosecha_disolve['area'] = gdf_cosecha_disolve.geometry.area / 10000
gdf_cosecha_disolve = gdf_cosecha_disolve.reset_index()
area_cosecha_disolve = gdf_cosecha_disolve['area'].sum()

In [19]:
area_cosecha_disolve

53918.56428932784

In [20]:
comprobar_idd_repetidos(gdf_cosecha_disolve)

Series([], Name: count, dtype: int64)

# COMPARAR CATASTRO - COSECHA

In [21]:
gdf_catastro_disolve['area'].sum()

54112.448550171815

In [22]:
gdf_catastro_disolve.head(3)

,idd,geometry,id,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,variedad,soca,...,textura,cultivo,zafra,financia,fs,fc,area,municipio,layer,path
0,1000|16213|L1.1,"POLYGON ((496536.407 8108251.116, 496650.951 8...",000010000000L1.1,1000,ROSALIA II--CKOLO ROSALIA,16213,ZAPATA SERON SIMON,L1.1,RB2,9,...,FY,canha,2024.0,None,2014-04-14,None,2.484536,None,catastroguabira,D:/DATOS 2024/MONITOREO QUEMA/catastroguabira.shp
1,1000|16213|L1.3,"POLYGON ((496629.283 8108328.873, 496743.683 8...",000010000000L1.3,1000,ROSALIA II--CKOLO ROSALIA,16213,ZAPATA SERON SIMON,L1.3,RB2,9,...,FY,canha,2024.0,None,2014-04-14,None,0.844639,None,catastroguabira,D:/DATOS 2024/MONITOREO QUEMA/catastroguabira.shp
2,1000|16213|L2.2,"POLYGON ((497138.550 8108758.112, 497221.635 8...",000010000000L2.2,1000,ROSALIA II--CKOLO ROSALIA,16213,ZAPATA SERON SIMON,L2.2,UCG9020,10,...,F,canha,2024.0,None,2013-04-10,None,3.241613,None,catastroguabira,D:/DATOS 2024/MONITOREO QUEMA/catastroguabira.shp


In [23]:
gdf_cosecha_disolve['area'].sum()

53918.56428932784

In [24]:
gdf_cosecha_disolve.head(3)

,idd,geometry,id,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,variedad,soca,...,textura,cultivo,zafra,financia,fs,fc,area,estado,fci,tipoc
0,1000|16213|L1.1,"POLYGON ((496536.407 8108251.116, 496650.951 8...",000010000000L1.1,1000,ROSALIA II--CKOLO ROSALIA,16213,ZAPATA SERON SIMON,L1.1,RB2,9,...,FY,canha,2024.0,None,2014-04-14,None,2.484536,sincosecha,0.0,cc
1,1000|16213|L1.3,"POLYGON ((496629.283 8108328.873, 496743.683 8...",000010000000L1.3,1000,ROSALIA II--CKOLO ROSALIA,16213,ZAPATA SERON SIMON,L1.3,RB2,9,...,FY,canha,2024.0,None,2014-04-14,None,0.844639,sincosecha,0.0,None
2,1000|16213|L2.2,"POLYGON ((497138.550 8108758.112, 497221.635 8...",000010000000L2.2,1000,ROSALIA II--CKOLO ROSALIA,16213,ZAPATA SERON SIMON,L2.2,UCG9020,10,...,F,canha,2024.0,None,2013-04-10,None,3.241613,sincosecha,0.0,cc


In [25]:
gdf_catastro_disolve['unidad_01'] = gdf_catastro_disolve['unidad_01'].astype(int)
gdf_cosecha_disolve['unidad_01'] = gdf_cosecha_disolve['unidad_01'].astype(int)

In [26]:
gdf_cat_pivot = gdf_catastro_disolve[['idd', 'area']]
gdf_cat_pivot

,idd,area
0,1000|16213|L1.1,2.484536
1,1000|16213|L1.3,0.844639
2,1000|16213|L2.2,3.241613
3,1000|16213|L3,4.265272
4,1000|16213|L4,9.282966
...,...,...
10626,9|18115|L2,1.714524
10627,9|18115|L3.1,1.639247
10628,9|18115|L3.2,1.772982
10629,9|18115|L4,4.286313


In [27]:
gdf_cos_pivot = gdf_cosecha_disolve[['idd', 'area']]
gdf_cos_pivot

,idd,area
0,1000|16213|L1.1,2.484536
1,1000|16213|L1.3,0.844639
2,1000|16213|L2.2,3.241613
3,1000|16213|L3,4.265272
4,1000|16213|L4,9.282966
...,...,...
10601,9|18115|L2,1.714524
10602,9|18115|L3.1,1.639247
10603,9|18115|L3.2,1.772982
10604,9|18115|L4,4.286313


In [28]:
df_cat_merged = pd.merge(gdf_cat_pivot, gdf_cos_pivot, on='idd', how='left')

In [29]:
gdf_cos_pivot

,idd,area
0,1000|16213|L1.1,2.484536
1,1000|16213|L1.3,0.844639
2,1000|16213|L2.2,3.241613
3,1000|16213|L3,4.265272
4,1000|16213|L4,9.282966
...,...,...
10601,9|18115|L2,1.714524
10602,9|18115|L3.1,1.639247
10603,9|18115|L3.2,1.772982
10604,9|18115|L4,4.286313


In [30]:
df_cat_merged.to_excel('merge.xlsx')